<a href="https://colab.research.google.com/github/TheRadDani/AI/blob/main/RAG_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Fetching API Key from Environment Variable

In [5]:
import os

# Read the API key from the uploaded file
with open('openai_token.txt', 'r') as file:
    openai_token = file.read().strip()

# Set the environment variable
os.environ['OPENAI_API_KEY'] = openai_token

In [6]:
import os
# Fetch the OpenAI API key from environment variables
api_key = os.environ.get('OPENAI_API_KEY')

In [ ]:
# check if api_key has loaded
print(api_key)

### Pipeline for Converting Raw Unstructured Data into a QA Chain

1. **Loading**: Initially, the data needs to be loaded. Unstructured data can be sourced from various platforms. Utilize the LangChain Integration Hub to explore the complete range of loaders. Each loader outputs the data as a LangChain Document.

2. **Splitting**: Text splitters segment the Documents into specified sizes.

3. **Storage**: A storage solution, often a vector store, is used to house and sometimes embed the splits.

4. **Retrieval**: The application fetches the splits from the storage, usually based on embeddings similar to the input question.

5. **Generation**: A Language Model (LLM) generates an answer using a prompt that incorporates both the question and the retrieved data.

6. **Conversation (Extension)**: To facilitate multi-turn conversations, Memory can be added to the QA chain.


![Q/A pipeline RAG](https://python.langchain.com/assets/images/qa_flow-9fbd91de9282eb806bda1c6db501ecec.jpeg)


In [ ]:
!pip install langchain

## Step 1: Loading the Document

In this initial step, we focus on loading the document into our system. This is a crucial phase as the quality of data loaded will significantly impact the subsequent stages of the pipeline. Here, you can use various loaders available in the LangChain Integration Hub to import your unstructured data as a LangChain Document.

In [9]:
from langchain.document_loaders import WebBaseLoader

# Initialize the WebBaseLoader with the URL of the document to be loaded
loader = WebBaseLoader("https://my.clevelandclinic.org/health/diseases/10946-cavities")

# Load the document and store it in the 'data' variable
data = loader.load()

In [11]:
# Display the content of the loaded document
print(data)
print(type(data))

[Document(page_content='Cavities (Tooth Decay): Symptoms, Causes & Treatment800.223.2273100 Years of Cleveland ClinicMyChartNeed Help?GivingCareersSearchClevelandClinic.orgFind A DoctorLocations & DirectionsPatients & VisitorsHealth LibraryInstitutes & DepartmentsAppointmentsHome/Health Library/Diseases & Conditions/CavitiesAdvertisementAdvertisementAdvertisementCavitiesCavities are holes, or areas of tooth decay, that form in your teeth surfaces. Causes include plaque buildup, eating lots of sugary snacks and poor oral hygiene. Treatments include dental fillings, root canal therapy and tooth extraction. The sooner you treat a cavity, the better your chance for a predictable outcome and optimal oral health.ContentsArrow DownOverviewSymptoms and CausesDiagnosis and TestsManagement and TreatmentPreventionOutlook / PrognosisLiving WithAdditional Common QuestionsContentsArrow DownOverviewSymptoms and CausesDiagnosis and TestsManagement and TreatmentPreventionOutlook / PrognosisLiving WithA

In [12]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.0 MB/s eta 0:00:00


## Step 2: Splitting the Document into Chunks

In this step, we aim to divide the loaded document into smaller, manageable chunks, also known as splits. This is essential for easier processing and retrieval in the subsequent stages of the pipeline.

In [13]:
import tiktoken

# Set up token encoding for the GPT-3.5 Turbo model
tiktoken.encoding_for_model('gpt-3.5-turbo')

<Encoding 'cl100k_base'>

In [14]:
tokenizer = tiktoken.get_encoding('cl100k_base')

# Define a function to calculate the token length of a given text
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tiktoken_len("Dentin decay: Dentin is the layer just beneath your tooth enamel.")

15

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter with specified parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = tiktoken_len
)

In [16]:
# Split the loaded document into smaller chunks
chunks = text_splitter.split_documents(data)

In [17]:
# Check the total number of chunks generated
len(chunks)

38

In [35]:
len(chunks[0].page_content)

470

## Step 3: Storing the Vector Embeddings in Vector Database

In this step, we will store the vector embeddings of the generated chunks into a vector database. This is crucial for efficient retrieval and further processing of the data.

1. **Database Storage**: To facilitate future retrieval of our document splits, it's essential to store them in a database.

2. **Embedding Model**: To convert our document splits into vector embeddings, we require an embedding model.

3. **Vector Store**: Finally, the vector embeddings and documents will be stored in a vector store. For this purpose, we will be using ChromaDB.


![img](https://python.langchain.com/assets/images/qa_data_load-70fac3ea6593b986613784dc056df21a.png)

In [ ]:
!pip install sentence-transformers

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# Specify the model name and additional arguments
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device' : 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

# Initialize HuggingFace Embeddings
hf = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

In [23]:
embed = hf.embed_documents(texts=['h','e'])

# Print the length of one of the embeddings to check its dimensions
print(len(embed[1]))

384


In [ ]:
# Install ChromaDB package using pip
!pip install chromadb

In [27]:
from langchain.vectorstores import Chroma

# Initialize Chroma vector database with chunks and HuggingFace embeddings
vectordb = Chroma.from_documents(chunks, hf)

In [28]:
# Perform a similarity search on the vector database
vectordb.similarity_search('bleeding gums', k=3)

[Document(page_content='sugary foods and drinks.Many adults also get cavities. Sometimes, new decay develops around the edges of cavities treated in childhood. Adults are also more likely to have receding gums. This condition exposes your teeth roots to plaque, which can cause cavities.AdvertisementCleveland Clinic is a non-profit academic medical center. Advertising on our site helps support our mission. We do not endorse non-Cleveland Clinic products or services. PolicySymptoms and CausesWhat are the signs of', metadata={'description': 'A cavity is a hole, or area of decay, in your tooth. Cavities form when acids in your mouth erode (wear down) your tooth enamel — your tooth’s hard, outer layer.', 'language': 'en', 'source': 'https://my.clevelandclinic.org/health/diseases/10946-cavities', 'title': 'Cavities (Tooth Decay): Symptoms, Causes & Treatment'}),
 Document(page_content='sodas, juice and milk). The bacteria convert these carbohydrates into acids.Bacteria, acid, food and saliva

## Step 4: Retrieve and Generate

In this step, we will retrieve the relevant chunks from the vector database and generate answers using a language model. This is the final step in the pipeline, bringing all the previous steps together to produce a coherent QA chain.

In [ ]:
!pip install openai

In [38]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Initialize a language model with ChatOpenAI
llm = ChatOpenAI(model_name= 'gpt-3.5-turbo', temperature=0.6)

# Initialize a RetrievalQA chain with the language model and vector database retriever
qa_chain = RetrievalQA.from_chain_type(llm, retriever= vectordb.as_retriever())

# Pass a query to the QA chain to generate an answer
qa_chain({'query' : 'How can I prevent cavity in my tooth?'})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
#Change the query to what you want to ask the LLM
query = 'What is enamel decay'

In [ ]:
qa_chain({'query' : query})

{'query': 'What is enamel decay',
 'result': 'Enamel decay refers to the breakdown or deterioration of the outermost layer of your tooth called enamel. This occurs due to the acids produced by bacteria in your mouth, which can be caused by poor oral hygiene, a diet high in sugary or acidic foods, or certain medical conditions. Enamel decay is the second stage of tooth decay and can lead to the formation of cavities or holes in the tooth. If left untreated, the decay can progress to affect the deeper layers of the tooth, leading to more severe dental problems.'}

##### Next Steps:

- Integrate prompt engineering

- Use Prompt Template from langchain